In [1]:

#!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py
#!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py
#!wget -q https://github.com/IntrotoML9103/Project/blob/main/final_utils.py

#!wget -qO- https://github.com/DM-GY-9103-2024F-H/9103-utils/releases/latest/download/lfw.tar.gz | tar xz

Data Processing

In [ ]:
import os
import random
from os import listdir, path

random.seed(42)

artist_names = [
    "Albrecht_Dürer",
    "Alfred_Sisley",
    "Amedeo_Modigliani",
    "Andy_Warhol",
    "Diego_Velazquez",
    "Edgar_Degas",
    "Francisco_Goya",
    "Frida_Kahlo",
    "Giotto_di_Bondone",
    "Gustav_Klimt",
    "Henri_Matisse",
    "Hieronymus_Bosch",
    "Joan_Miro",
    "Kazimir_Malevich",
    "Leonardo_da_Vinci",
    "Marc_Chagall",
    "Mikhail_Vrubel",
    "Pablo_Picasso",
    "Paul_Gauguin",
    "Paul_Klee",
    "Peter_Paul_Rubens",
    "Pierre_Auguste_Renoir",
    "Pieter_Bruegel",
    "Raphael",
    "Rembrandt",
    "Rene_Magritte",
    "Salvador_Dali",
    "Sandro_Botticelli",
    "Titian",
    "Vincent_van_Gogh"
]

artists_dict = {name:index  for index, name in enumerate(artist_names)}

file_names = sorted([f for f in listdir("imgs") if f.endswith(".jpg")])

print(len(file_names))

random.shuffle(file_names)

train_files = file_names[:2424]
test_files = file_names[2424:]

print(len(train_files))
print(len(test_files))

train_list = [] 
train_label = [] # get name from file - artists_dict map the name to the number - e.g: index Durer - returns 0 

for:
img = open_image(dirname+fname)
img_t = Tensor(img.pixels).reshape(224,224,3).permute(2,0,1)






test_list = []
test_label = []






3030
2424
606
